In [1]:
import os
import copy
import pickle
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

# Global variables
MAX_NO_OF_SPEAKERS = 9
EMOTIONS = ['anger','joy','surprise','neutral','sadness','disgust','fear','contempt']

model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

/home/sgarc/SemEval-2023-Task-10/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_csv = pd.read_json("EDiReF-Train-Data/Task 1/MaSaC_train_erc.json")

In [3]:
train_csv

,episode,speakers,utterances,emotions
0,roseshantstobeadopted.wmv,"[maya, indu, rosesh, indu, maya, indu, rosesh,...",[kya kya bhar ke rakha hai indravadan ne is gh...,"[disgust, contempt, anger, neutral, neutral, c..."
1,roseshantstobeadopted.wmv,"[maya, rosesh, indu, rosesh, maya, maya, indu,...","[rosesh beta happy diwali!, you bet momma., ro...","[joy, neutral, joy, sadness, neutral, joy, neu..."
2,roseshantstobeadopted.wmv,"[maya, indu, maya, indu, maya, maya, indu, may...","[radha bai! vitthal! monisha!, maya suno, ye k...","[neutral, fear, disgust, neutral, contempt, ne..."
3,roseshantstobeadopted.wmv,"[sahil, monisha, sahil, monisha, sahil, monish...","[rosesh chup ho jao. monisha pani lekar aao!, ...","[anger, neutral, neutral, neutral, neutral, ne..."
4,roseshantstobeadopted.wmv,"[rosesh, maya, rosesh, maya, rosesh, maya, ros...","[momma aap mujhse dur hi rahe to behtar hoga, ...","[sadness, neutral, contempt, neutral, sadness,..."
...,...,...,...,...
338,radhabaileaves,"[sahil, radha bai, vitthal, radha bai, sahil, ...",[radha bhai radha bhai please aap rona band ki...,"[neutral, sadness, sadness, sadness, neutral, ..."
339,radhabaileaves,"[maya, indu, rosesh, indu, maya, indu, maya, i...","[indravadan! rosesh ka play hit ho gaya!, achc...","[joy, joy, neutral, neutral, neutral, joy, joy..."
340,radhabaileaves,"[maya, monish, maya, sahil, maya, sahil, monis...",[monisha tumne un logon ko bhadkake acha nahin...,"[anger, neutral, neutral, neutral, contempt, a..."
341,radhabaileaves,"[ila ben, maya, indu, ila ben, indu, maya, ila...","[maya ye hai sita aur geeta, ye akeli hi seeta...","[neutral, surprise, joy, neutral, joy, neutral..."


In [6]:
def one_hot_emotions(dts: pd.DataFrame) -> list:
    emotions_dummies = pd.get_dummies(EMOTIONS)
    return [[emotions_dummies[x_1] for x_1 in x_0] for x_0 in dts]

In [7]:
max([len(set(x)) for x in train_csv["speakers"]])

9

In [8]:
def one_hot_speakers(dts: pd.DataFrame) -> list:
    tmp = []
    for conversation in dts:
        speakers, idxs = np.unique(conversation, return_inverse=True)
        tmp.append(np.eye(len(speakers), MAX_NO_OF_SPEAKERS)[idxs])
    return tmp

In [9]:
def generate_sentence_embeddings(dts: pd.DataFrame, model: SentenceTransformer) -> list:
    if not os.path.exists('df_sent_task1.pkl'):
        sentence_embeddings = [copy.deepcopy(model.encode(utt)) for utt in dts]
        with open('df_sent_task1.pkl', 'wb') as f:
            pickle.dump(sentence_embeddings, f)
    else:
        with open('df_sent_task1.pkl', 'rb') as f:
            sentence_embeddings = pickle.load(f)
    
    return sentence_embeddings

In [10]:
generate_sentence_embeddings(train_csv["utterances"], model)

: 